In [446]:
# Import packages and read-in files

import numpy as np
import pandas as pd
import xpress as xp
from datetime import datetime, timedelta
import os
import math 

# Read in files using the explicitly defined base path
ch_0_conversion_rates = pd.read_csv('channel_0_conversion_rates.csv')
ch_0_schedule = pd.read_csv('channel_0_schedule.csv')
ch_1_conversion_rates = pd.read_csv('channel_1_conversion_rates.csv')
ch_1_schedule = pd.read_csv('channel_1_schedule.csv')
ch_2_conversion_rates = pd.read_csv('channel_2_conversion_rates.csv')
ch_2_schedule = pd.read_csv('channel_2_schedule.csv')
ch_A_schedule = pd.read_csv('channel_A_schedule.csv')
movies_df = pd.read_csv('movie_database.csv')

In [447]:
# To use right xpress and get rid of unnecessary error codes
xp.init('C:/xpressmp/bin/xpauth.xpr')

In [448]:
pd.options.mode.copy_on_write = True

In [449]:
# FORMATING
# Convert 'Date-Time' columns to datetime format
date_cols = ['Date']

for df in [ch_0_conversion_rates, ch_0_schedule, ch_1_conversion_rates, ch_1_schedule,
           ch_2_conversion_rates, ch_2_schedule, ch_A_schedule]:
    df['Date'] = pd.to_datetime(df['Unnamed: 0'])
    df.set_index('Date', inplace=True)
    df.drop('Unnamed: 0', axis=1, inplace = True)
   

# Convert 'Release Date' in movie_database to datetime
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'])

# Fill missing values if necessary
movies_df.fillna(0, inplace=True)


In [450]:
# Make 30-min slots
# Slot duration 30 minutes
slot_duration = 30  # minutes
slots_needed = (movies_df['runtime_with_ads'] / slot_duration).apply(lambda x: int(x)).astype(int)
movies_df['slots_needed'] = slots_needed

In [451]:
# Check for duplicate movie titles
duplicate_titles = movies_df[movies_df.duplicated(subset=['title'], keep=False)]
if not duplicate_titles.empty:
    print("Duplicate movie titles found:")
    print(duplicate_titles['title'])
else:
    print("No duplicate movie titles found.")

Duplicate movie titles found:
4                 The Avengers
17                     Titanic
76               The Lion King
105       Beauty and the Beast
149        Alice in Wonderland
                 ...          
5748              Midnight Sun
5761                The Island
5773            The Shaggy Dog
5855    Fun with Dick and Jane
5879        The Perfect Weapon
Name: title, Length: 258, dtype: object


In [452]:
# Make broadcast date for any number of days

# Choose the number of days you're make a schedule for 
    # this can  be used in the functions later
num_days = 7

# Define the broadcasting start and end dates
broadcast_start_date = datetime.strptime("2024-10-01", "%Y-%m-%d")
broadcast_end_date = broadcast_start_date + timedelta(days=num_days)  # 7 days including start date

# Define daily broadcast start and end times
daily_broadcast_start_time = timedelta(hours=7, minutes=0)
daily_broadcast_end_time = timedelta(hours=23, minutes=30)

# Generate all time slots over the date range
time_slots = []
current_date = broadcast_start_date
while current_date <= broadcast_end_date:
    # Set the start and end times for the current day
    day_start = datetime.combine(current_date.date(), datetime.min.time()) + daily_broadcast_start_time
    day_end = datetime.combine(current_date.date(), datetime.min.time()) + daily_broadcast_end_time

    current_time = day_start
    while current_time <= day_end:
        time_slots.append(current_time)
        current_time += timedelta(minutes=slot_duration)

    # Move to the next day
    current_date += timedelta(days=1)

# Create mappings between time slots and indices
time_to_index = {t: idx for idx, t in enumerate(time_slots)}
index_to_time = {idx: t for idx, t in enumerate(time_slots)}

In [453]:
# get averagae view counts and prime time factor for each 30 minute time slot
ch_A_schedule_30min = ch_A_schedule.resample('30min').mean()

# ch_A_schedule_30min.head()

In [454]:
# sum base viewership across each row (for total base viewership of each slot)
ch_A_schedule_30min['total_view_count'] = np.nan
for i in ch_A_schedule_30min:
    ch_A_schedule_30min['total_view_count'] = ch_A_schedule_30min['children_baseline_view_count'] + ch_A_schedule_30min['adults_baseline_view_count'] + ch_A_schedule_30min['retirees_baseline_view_count']

# ch_A_schedule_30min.head(1)

In [455]:
ch_A_schedule_30min.reset_index(inplace = True)
# ch_A_schedule_30min.head(2)

In [456]:
# # make objective function expression
# objective_terms = []
# for row in movies_small:
#     var = x[m,t]
#     coeff = ch_A_schedule_30min.loc[t,-1]
#     if coeff != 0:
#         objective_terms.append(coeff*var)
# prob.setObjective(xp.Sum(objective_terms), sense=xp.maximize)


In [457]:
# movies_small.head(1)

In [458]:
# Make a smaller movies dataframe so it doesn't take so long
movies_small = movies_df.head(300)
movies_small.set_index('title', inplace = True)

In [459]:
# movies_small.loc['Inception', 'children_scaled_popularity']

In [460]:
# ch_A_schedule_30min.head(1)

In [461]:
# ch_A_schedule_30min['children_baseline_view_count'].iloc[0]

In [462]:
# # get views per movie per time slot
# population = 1_000_000
# pop_demogs = ['children_scaled_popularity', 'adults_scaled_popularity', 'retirees_scaled_popularity']
# view_demogs = ['children_baseline_view_count', 'adult_baseline_view_count', 'retirees_baseline_view_count']
# def view_per_mov_time(movie_dataframe, our_channel, population, pop_demographics, view_demographics):
#     """ Calculate baseline viewership for movie m at time slot t"""
#     views = x[m,t]*xp.Sum(population*movie_dataframe[k].loc[m]*our_channel[j].loc[m]  for k in pop_demogs for j in view_demogs)
#     return views

In [463]:
# def view_per_mov_time(movies):
#     """ baseline viewership """
#     views = x[m,t]*xp.Sum(population * movies['children_scaled_popularity'].loc[m]

In [464]:
# # make u(m,t) to give view counts of movie m at time t
# u[m,t] <= (xp.Sum(view_per_m_t(x, m, t, movie_dataframe, our_channel, population. pop_demographics, view_demographics) for m in movies.index for t in T))

In [465]:
 movies_small.head(1)

,vote_average,vote_count,release_date,revenue,runtime,budget,popularity,genres,n_ad_breaks,runtime_with_ads,scaled_popularity,children_scaled_popularity,adults_scaled_popularity,retirees_scaled_popularity,slots_needed
title,,,,,,,,,,,,,,,
Inception,8.364,34495,2010-07-15,825532764,148,160000000,83.952,"['Action', 'Science Fiction', 'Adventure']",5,180.0,0.972027,0.777622,0.972027,0.388811,6


In [466]:
ch_A_schedule_30min.head(1)

,Date,children_baseline_view_count,adults_baseline_view_count,retirees_baseline_view_count,prime_time_factor,total_view_count
0,2024-10-01 07:00:00,0.019383,0.022296,0.003611,1.0,0.04529


In [467]:
# pop = 1_000_000
# pop_demogs = ['children_scaled_popularity', 'adults_scaled_popularity', 'retirees_scaled_popularity']
# view_demogs = ['children_baseline_view_count', 'adults_baseline_view_count', 'retirees_baseline_view_count']

In [486]:
# Get viewership counts for movie m in time slot t

def get_views(movies, our_channel):
    #ans = xp.Sum(movies.loc[m,k]*our_channel[j].iloc[t] for k in pop_demos for j in view_demos for m in movies.index for t in T)
    for n in movies.index:
        for c in our_channel.index:
            ans = ((movies['children_scaled_popularity'].loc[n]*our_channel['children_baseline_view_count'].iloc[c])
                   + (movies['adults_scaled_popularity'].loc[n]*our_channel['adults_baseline_view_count'].iloc[c])
                   + (movies['retirees_scaled_popularity'].loc[n]*our_channel['retirees_baseline_view_count'].iloc[c]) )*1_000_000
    ans = math.floor(ans)
    print(ans)

In [469]:
# get_views(movies = movies_small.head(1), our_channel = ch_A_schedule_30min.head(1))

In [487]:
# Scheduling Optimizer Model
objective_terms = []
def model(T, movies, our_channel):
    prob = xp.problem(name="Movie_Scheduling_Problem")
    
    # Decision Variables
    # if movie m in shown in time slot t
    x = {(m,t): xp.var(vartype=xp.binary, name='x{0}_{1}'.format(m,t)) 
         for t in T for m in movies.index}
    prob.addVariable(x)
    
    y = {(m): xp.var(vartype=xp.binary, name='y{0}'.format(m)) for m in movies.index}
    prob.addVariable(y)

    # start time of movie m 
    s = {(m): xp.var(vartype=xp.integer, name='s{0}'.format(m)) for m in movies.index}
    prob.addVariable(s)

    # end time of movie m 
    e = {(m): xp.var(vartype=xp.integer, name='e{0}'.format(m)) for m in movies.index}
    prob.addVariable(e)
    decision_vars = [x,y,s,e]

    # view count of movie m at time t
    u = {(m,t): xp.var(vartype=xp.integer, name='u_{0}_{1}'.format(m,t)) for t in T for m in movies.index}
    

    # Constraints
    # for t in T:
        # big M 
    M = len(time_slots)
    T_end = len(time_slots) -1 # last time slot index
    
    # movie duration 
    prob.addConstraint(xp.Sum(x[m,t] for t in T) == movies.loc[m, 'slots_needed']*y[m] for m in movies.index)

    # has to be a movie in every slot
    prob.addConstraint(xp.Sum(x[m,t] for m in movies.index) == 1 for t in T)

    # end time
    # prob.addConstraint(e[m] == t*xp.Sum(x) for m in movie.index())

    # end time limit
    prob.addConstraint((t+1)*x[m,t] <= e[m] for m in movies.index for t in T)

    # start time limit
    #prob.addConstraint(s[m] <= (t*x[m,t]) for m in movie.index() fot t in T)

    # latest time a movie can start and still show full movie 
    prob.addConstraint(s[m] <= t*x[m,t] + (1-x[m,t])*M for m in movies.index for t in T)

    # end-start = movie duration
    prob.addConstraint(e[m]-s[m] == movies.loc[m, 'slots_needed']*y[m] for m in movies.index)

    # last movie time?
    prob.addConstraint(s[m] + movies.loc[m, 'slots_needed'] -1 <= T_end for m in movies.index)

    # make u(m,t) to give view counts of movie m at time t
    prob.addConstraint(x[m,t]*xp.Sum(get_views(movies, our_channel) for m in movies.index for c in our_channel.index) >= u[m,t])

  # def get_views(movies, our_channel):
  #   #ans = xp.Sum(movies.loc[m,k]*our_channel[j].iloc[t] for k in pop_demos for j in view_demos for m in movies.index for t in T)
  #   for m in movies.index:
  #       for c in our_channel.index:
  #           ans = ((movies['children_scaled_popularity'].loc[m]*our_channel['children_baseline_view_count'].iloc[c])
  #                  + (movies['adults_scaled_popularity'].loc[m]*our_channel['adults_baseline_view_count'].iloc[c])
  #                  + (movies['retirees_scaled_popularity'].loc[m]*our_channel['retirees_baseline_view_count'].iloc[c]) )*1_000_000
  #   ans = math.floor(ans)
  #   print(ans)

    # make objective function expression
    prob.setObjective(xp.Sum(u[m,t] for m in movies.index for t in T), sense = xp.maximize)

    return prob, decision_vars

In [488]:
# Making schedule layout
def get_time(slot_index):
    return time_slots[slot_index].time()

used_movie_ids = []
def get_sched(prob, movies, decision_vars): 
    
    x = decision_vars[0]
    y = decision_vars[1]
    s = decision_vars[2]
    e = decision_vars[3]
    
    scheduled_movies = []
    used_movie_ids = []

    for m in movies.index:
    # Retrieve the solution value for y[m_idx]
        y_value = prob.getSolution(y[m])
        
        if y_value > 0.5:  # Movie is scheduled
            used_movie_ids.append(m)
            
            # # Retrieve solution values for s[m_idx] and e[m_idx]
            start_slot = int(prob.getSolution(s[m]))
            end_slot = int(prob.getSolution(e[m]))
            
            # # # # Convert slot indices to actual times
            start_time = get_time(start_slot)
            end_time = get_time(end_slot)
    
            # # Append the scheduled movie details
            scheduled_movies.append({
                'Movie Title': m,
                'Start Slot': start_slot,
                'Start Time': start_time.strftime('%H:%M'),
                'End Slot': end_slot,
                'End Time': end_time.strftime('%H:%M')
            })
    schedule_df = pd.DataFrame.from_dict(scheduled_movies)
    schedule_df.sort_values(['Start Slot'], axis = 0, inplace = True)
    print(schedule_df)
    return scheduled_movies, used_movie_ids
    

In [489]:
# Putting it all together

def movie_sched(number_days, movies, our_channel):
    total_schedule = []
    for k in range(number_days): 
        if k == 0:
            T = range(0, (k+1)*33)

            prob, decision_vars =  model(T,movies, our_channel)
            
            prob.solve()
            
            scheduled_movies, used_movie_ids = get_sched(prob,movies,decision_vars)
            
            for i in used_movie_ids:
                movies.drop(i, inplace = True)
            total_schedule.append(scheduled_movies)
            
        if k > 0:
            T = range(k*34, (k*34)+33)

            prob, decision_vars =  model(T,movies, our_channel)
            
            prob.solve()
            
            scheduled_movies, used_movie_ids = get_sched(prob,movies,decision_vars)
            
            for i in used_movie_ids:
                movies.drop(i, inplace = True)
            total_schedule.append(scheduled_movies)
            
    return scheduled_movies

In [490]:
# pop = 1_000_000
# pop_demogs = ['children_scaled_popularity', 'adults_scaled_popularity', 'retirees_scaled_popularity']
# view_demogs = ['children_baseline_view_count', 'adult_baseline_view_count', 'retirees_baseline_view_count']

In [491]:
# try for 7 days
movie_sched(number_days = 2, movies = movies_small, our_channel = ch_A_schedule_30min)

NameError: name 'm' is not defined